In [171]:
import numpy as np
import pandas as pd

from tqdm import tqdm

In [172]:
import absl.logging
absl.logging.set_verbosity(absl.logging.ERROR)

In [173]:
import tensorflow as tf

<h3>Read files</h3>

In [174]:
with open('../data/models/tmp/test.npy', 'rb') as f:
    test = np.load(f, allow_pickle=True)

In [175]:
with open('../data/models/tmp/vocab.npy', 'rb') as f:
    vocab = np.load(f, allow_pickle=True)
    categ = np.load(f, allow_pickle=True)

In [201]:
item_mask = pd.Series(vocab).apply(lambda x: x[:2] == 'ID').values.astype(float)
item_mask = np.insert(item_mask, 0, 0)

In [216]:
item_target = pd.Series(test[:, -1]).apply(lambda x: x[:2] == 'ID').values

In [202]:
%%time
pretrained = tf.keras.models.load_model('../data/models/model.h5', custom_objects={}) # load model

CPU times: user 995 ms, sys: 24.8 ms, total: 1.02 s
Wall time: 1.02 s


In [203]:
lookup = pretrained.get_layer('item_embedding').get_layer('lookup')

In [204]:
def get_model():
    
    freeze = pretrained.output
    
    output = tf.multiply(freeze, item_mask)
    
    return tf.keras.Model(inputs=pretrained.input, outputs=output, name="model")

In [205]:
model = get_model()

In [254]:
model.compile(
    metrics=[
        tf.keras.metrics.SparseTopKCategoricalAccuracy(k=int(item_mask.sum() // 2), name='auc'), # auc
        tf.keras.metrics.SparseTopKCategoricalAccuracy(k=1, name='top1acc'),
        tf.keras.metrics.SparseTopKCategoricalAccuracy(k=10, name='top10acc')
    ])

In [255]:
_ = model.evaluate(test[:, :-1][item_target], lookup(test[:, -1][item_target]))

342/342 [==============================] - 2s 4ms/step - loss: 0.0000e+00 - auc: 0.9206 - top1acc: 0.4170 - top10acc: 0.5086
